# State Roads/Rail/Runway Shapefile Decoding

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))


## Install
import shapefile
import geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from fsUtils import isFile
from ioUtils import showSize, saveJoblib
from strUtils import nice as fixStr
from geoUtils import *
from geospatialUtils import saveGeoData, getBB, rmZipDir, unZipFile, getGeos
import pickle
from glob import glob
from os.path import basename,splitext,join
from collections import Counter
from roadsUtils import getRoadsPoints, getRoadsData

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.5 |Anaconda custom (x86_64)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-11-19 20:36:59.367897


In [2]:
# Global Params
basedir="/Users/tgadf/Downloads/stateshapes"

In [3]:
if False:
    zipfiles = glob(join(basedir, "*Del*.zip"))
    print("Found {0} zip files".format(len(zipfiles)))
    for zipname in zipfiles:
        unZipFile(zipname, debug=True, test=False)

In [5]:
from collections import Counter
from os.path import basename, dirname, exists, splitext, isdir, basename
from glob import glob
import zipfile
from os import mkdir
from shutil import rmtree

prec=7
shapevals = ["Trans_AirportPoint", "Trans_AirportRunway", "Trans_RailFeature", "Trans_RoadSegment2", "Trans_TrailSegment", "Trans_RoadSegment", "Trans_RoadSegment3"]
shapevals = ["Trans_AirportPoint", "Trans_AirportRunway"]

states = ["11", "17", "24", "51"]
states = [str(x) for x in range(1,80)]
#states = ["17"]
basevals = ["TrailSegment", "RailFeature", "RoadSegment"]
basevals = ["RoadSegment"]
basevals = ["AirportRunway"]
#basevals = ["RailFeature"]



show=False
for state in states:
    print("State ----> {0}".format(state))
    
    zipfiles = glob(join(basedir, "TRAN_{0}_*_GU_STATEORTERRITORY.zip".format(state)))
    print("Found {0} zip files".format(len(zipfiles)))
    if len(zipfiles) == 0:
        continue
    elif len(zipfiles) == 1:
        zipname = zipfiles[0]
        rmZipDir(zipname, test=False)
    unZipFile(zipname, test=False, debug=True)
    
    newdirname = None
    for baseval in basevals:
        print("  Type ----> {0}".format(baseval))
        
        scntr = Counter()
        remainCntr = Counter()

        shapeData   = {}
        geoShapeMap = {}
        totalgeos   = 0
        keepFrac    = 0
        totalFrac   = 0
        
        shapefiles  = glob(join(basedir, "TRAN_{0}_*_GU_STATEORTERRITORY".format(state), "Shape", "*{0}*.shp".format(baseval)))

        for sfile in shapefiles:
            shapename = basename(splitext(sfile)[0])
            shapeval  = shapename.replace("Trans_", "")
            print(sfile)
            print(shapename)
            print(shapeval)
            
            sf = shapefile.Reader(sfile)
            #except:
            #    print("Could not open shape file: {0}".format(shapefile))
            #    continue
            fields = sf.fields
            shapes = sf.shapes()
            Nshapes = len(shapes)
            ngeos = 0

            start,cmt = clock("Analyzing {0}\t{1}\t{2}".format(state, shapeval, Nshapes))

            itype, iint = getRoadsPoints(sf, shapeval)


            irec = -1
            ngeos = 0
            for shapeRec in sf.iterShapeRecords():
                totalFrac += 1
                keep = True
                irec += 1

                if show:
                    print(len(shapeRec.shape.points), shapeRec.record)
                    #continue

                ## Record
                record = shapeRec.record
                #[(('DeletionFlag', 'C', 1, 0), '1'), (['TEMP', 'C', 200, 0], '4'), (['TNMFRC', 'C', 200, 0], '636087b7-e6f3-478b-a98a-7ec1e000e639'), 
                # (['PERMANENT_', 'C', 200, 0], '110368847020'), (['SOURCE_FEA', 'C', 200, 0], '{1D13E0B6-E3E3-4631-B37C-ABEE7881E78E}'), 
                # (['SOURCE_DAT', 'C', 200, 0], '2016 BAS Tiger/Line Shapefiles Release'),
                # (['SOURCE_D00', 'C', 200, 0], 'U.S Department of Commerce, U.S. Census Bureau, Geography Division'), 
                # (['SOURCE_ORI', 'C', 200, 0], '20160420170929'), (['LOADDATE', 'C', 200, 0], ''), (['INTERSTATE', 'C', 200, 0], ''), 
                # (['INTERSTA00', 'C', 200, 0], ''), (['INTERSTA01', 'C', 200, 0], ''), (['INTERSTA02', 'C', 200, 0], ''), (['US_ROUTE', 'C', 200, 0], ''), 
                # (['US_ROUTE_A', 'C', 200, 0], ''), (['US_ROUTE_B', 'C', 200, 0], ''), (['US_ROUTE_C', 'C', 200, 0], ''), (['STATE_ROUT', 'C', 200, 0], ''), 
                # (['STATE_RO00', 'C', 200, 0], ''), (['STATE_RO01', 'C', 200, 0], ''), (['STATE_RO02', 'C', 200, 0], ''), (['COUNTY_ROU', 'C', 200, 0], ''), 
                # (['FEDERAL_LA', 'C', 200, 0], '01023'), (['STCO_FIPSC', 'C', 200, 0], 'Gin Creek Rd'), (['FULL_STREE', 'C', 200, 0], 'S1400'), 
                # (['MTFCC_CODE', 'C', 200, 0], '0.0356307362677334')]                    
                #print(list(zip(fields,record)))
                #1/0


                recData = getRoadsData(shapeval, record, itype, iint, scntr)
                #print(recData)
                geoid = recData[0] ## Not currently using this
                rname = recData[1] ## Not currently using this

                if shapeval.startswith("TrailSegment"):
                    catID = "Trail"
                elif shapeval.startswith("RailFeature"):
                    catID = "Rail"
                elif shapeval.startswith("AirportRunway"):
                    catID = "Runway"
                elif shapeval.startswith("RoadSegment"):
                    if recData[3]:
                        catID = "Interstate"
                    elif recData[4]:
                        catID = "USRte"
                    elif recData[5]:
                        catID = "StateRte"
                    elif recData[6]:
                        catID = "Highway"
                    elif recData[7]:
                        catID = "MajorRd"
                    elif recData[8]:
                        catID = "Connection"
                    else:
                        catID = "Road"
                        #catID = None

                ## Shape
                shape = shapeRec.shape


                ## Geos
                geos       = getGeos(shape, prec=prec, linear=True, returnKeys=True)
                if False:
                    print(len(shape.points))
                    print(len(geos))
                    if len(geos) > 20:
                        print(len(geos))
                        print(record)
                        1/0

                if irec % 10000 == 0:
                    print(fixStr(str(irec), 10), fixStr(str(catID), 10), fixStr(str(rname),30), fixStr(str(len(shape.points)),10), fixStr(str(len(geos)), 10), fixStr(str(ngeos), 10), fixStr(str(totalgeos), 10))

                if catID is None:
                    continue

                keepFrac  += 1

                if totalFrac % 50000 == 0 and totalFrac > 0:
                    print("\tKeeping {0} of {1} = {2}".format(keepFrac, totalFrac, round(100*keepFrac/totalFrac,1)))
                    
                    
                lgeos      = len(geos)
                ngeos     += lgeos
                totalgeos += lgeos

                geoid = catID
                rname = catID
                if shapeData.get(catID) is None:
                    shapeData[catID] = {}
                    shapeData[catID][geoid]   = {"Name": rname}
                    geoShapeMap[catID] = {}
                    geoShapeMap[catID][geoid] = set()
                for geo in geos:
                    geoShapeMap[catID][geoid].add(geo)


            print("Found {0} geos from {1}-{2}".format(ngeos, state, shapeval))


    for catID,catData in shapeData.items():
        shapeval = catID
        ngeos = len(geoShapeMap[catID][catID])
        print("Found {0} of type {1}/{2}".format(ngeos,baseval,shapeval))
        saveGeoData(shapeData[catID], geoShapeMap[catID], 0, ngeos, "{0}/roads/{1}-{2}-{3}-{4}".format(basedir, baseval, shapeval, state, prec))

    rmZipDir(zipname)

State ----> 1
Found 1 zip files
Unzipping /Users/tgadf/Downloads/stateshapes/TRAN_1_Alabama_GU_STATEORTERRITORY.zip to /Users/tgadf/Downloads/stateshapes/TRAN_1_Alabama_GU_STATEORTERRITORY
  Type ----> AirportRunway
/Users/tgadf/Downloads/stateshapes/TRAN_1_Alabama_GU_STATEORTERRITORY/Shape/Trans_AirportRunway.shp
Trans_AirportRunway
AirportRunway
Current Time is Mon Nov 19, 2018 20:40:46 for Analyzing 1	AirportRunway	204
0          Runway     None                           5          7          0          0         
Found 2207 geos from 1-AirportRunway
Found 2179 of type AirportRunway/Runway


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-1-7-data.p


There are 2179 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-1-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/stateshapes/TRAN_1_Alabama_GU_STATEORTERRITORY
State ----> 2
Found 1 zip files
Unzippi

  Type ----> AirportRunway
/Users/tgadf/Downloads/stateshapes/TRAN_12_Florida_GU_STATEORTERRITORY/Shape/Trans_AirportRunway.shp
Trans_AirportRunway
AirportRunway
Current Time is Mon Nov 19, 2018 20:43:07 for Analyzing 12	AirportRunway	623
0          Runway     None                           5          25         0          0         
Found 6947 geos from 12-AirportRunway
Found 6743 of type AirportRunway/Runway


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-12-7-data.p


There are 6743 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-12-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/stateshapes/TRAN_12_Florida_GU_STATEORTERRITORY
State ----> 13
Found 1 zip files
Unzipping /Users/tgadf/Downloads/stateshapes/TRAN_13_Georgia_GU_STATEORTERRITORY.zip to /Users/tgadf/Downloads/stateshapes/TRAN_13_Georgia_GU_STATEORTERRITORY
  Type ----> AirportRunway
/Us

Unzipping /Users/tgadf/Downloads/stateshapes/TRAN_22_Louisiana_GU_STATEORTERRITORY.zip to /Users/tgadf/Downloads/stateshapes/TRAN_22_Louisiana_GU_STATEORTERRITORY
  Type ----> AirportRunway
/Users/tgadf/Downloads/stateshapes/TRAN_22_Louisiana_GU_STATEORTERRITORY/Shape/Trans_AirportRunway.shp
Trans_AirportRunway
AirportRunway
Current Time is Mon Nov 19, 2018 20:45:13 for Analyzing 22	AirportRunway	264
0          Runway     None                           5          13         0          0         
Found 2453 geos from 22-AirportRunway
Found 2396 of type AirportRunway/Runway


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-22-7-data.p


There are 2396 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-22-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/stateshapes/TRAN_22_Louisiana_GU_STATEORTERRITORY
State ----> 23
Found 1 zip files
Unzipping /Users/tgadf

State ----> 31
Found 1 zip files
Unzipping /Users/tgadf/Downloads/stateshapes/TRAN_31_Nebraska_GU_STATEORTERRITORY.zip to /Users/tgadf/Downloads/stateshapes/TRAN_31_Nebraska_GU_STATEORTERRITORY
  Type ----> AirportRunway
/Users/tgadf/Downloads/stateshapes/TRAN_31_Nebraska_GU_STATEORTERRITORY/Shape/Trans_AirportRunway.shp
Trans_AirportRunway
AirportRunway
Current Time is Mon Nov 19, 2018 20:47:18 for Analyzing 31	AirportRunway	265
0          Runway     None                           5          11         0          0         
Found 2557 geos from 31-AirportRunway
Found 2467 of type AirportRunway/Runway


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-31-7-data.p


There are 2467 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-31-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/stateshapes/TRAN_31_Nebraska_GU_STATEORTERRITORY
State ----> 32
Found 1 zip

  Type ----> AirportRunway
/Users/tgadf/Downloads/stateshapes/TRAN_40_Oklahoma_GU_STATEORTERRITORY/Shape/Trans_AirportRunway.shp
Trans_AirportRunway
AirportRunway
Current Time is Mon Nov 19, 2018 20:49:47 for Analyzing 40	AirportRunway	351
0          Runway     None                           5          24         0          0         
Found 3193 geos from 40-AirportRunway
Found 3123 of type AirportRunway/Runway


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-40-7-data.p


There are 3123 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-40-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/stateshapes/TRAN_40_Oklahoma_GU_STATEORTERRITORY
State ----> 41
Found 1 zip files
Unzipping /Users/tgadf/Downloads/stateshapes/TRAN_41_Oregon_GU_STATEORTERRITORY.zip to /Users/tgadf/Downloads/stateshapes/TRAN_41_Oregon_GU_STATEORTERRITORY
  Type ----> AirportRunway
/Us

  Type ----> AirportRunway
/Users/tgadf/Downloads/stateshapes/TRAN_50_Vermont_GU_STATEORTERRITORY/Shape/Trans_AirportRunway.shp
Trans_AirportRunway
AirportRunway
Current Time is Mon Nov 19, 2018 20:52:30 for Analyzing 50	AirportRunway	64
0          Runway     None                           5          6          0          0         
Found 476 geos from 50-AirportRunway
Found 463 of type AirportRunway/Runway


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-50-7-data.p


There are 463 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-50-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/stateshapes/TRAN_50_Vermont_GU_STATEORTERRITORY
State ----> 51
Found 1 zip files
Unzipping /Users/tgadf/Downloads/stateshapes/TRAN_51_Virginia_GU_STATEORTERRITORY.zip to /Users/tgadf/Downloads/stateshapes/TRAN_51_Virginia_GU_STATEORTERRITORY
  Type ----> AirportRunway
/User

  Type ----> AirportRunway
/Users/tgadf/Downloads/stateshapes/TRAN_69_Northern_Mariana_Islands_GU_STATEORTERRITORY/Shape/Trans_AirportRunway.shp
Trans_AirportRunway
AirportRunway
Current Time is Mon Nov 19, 2018 20:53:45 for Analyzing 69	AirportRunway	5
0          Runway     None                           5          13         0          0         
Found 86 geos from 69-AirportRunway
Found 86 of type AirportRunway/Runway


There are 1 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-69-7-data.p


There are 86 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-69-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/stateshapes/TRAN_69_Northern_Mariana_Islands_GU_STATEORTERRITORY
State ----> 70
Found 0 zip files
State ----> 71
Found 0 zip files
State ----> 72
Found 1 zip files
Unzipping /Users/tgadf/Downloads/stateshapes/TRAN_72_Puerto_Rico_GU_STATEORTERRITORY.zip to /User

# Combine Into Single Data

In [9]:
import pickle
from glob import glob
import string
from collections import Counter
from os.path import basename,splitext
vals     = glob("{0}/roads/*-{1}-geos.p".format(basedir, prec))
basevals = set([splitext(basename(x))[0].split('-')[0] for x in vals])
mtypes   = set([splitext(basename(x))[0].split('-')[1] for x in vals])
states   = set([splitext(basename(x))[0].split('-')[2] for x in vals])
try:
    mtypes.remove('Connection')
    mtypes.remove('Trail')
except:
    pass
#mtypes.remove('Rail')
try:
    basevals.remove('TrailSegment')
except:
    pass
#basevals.remove('RailFeature')

#print(mtypes)
#print(basevals)
mtypes={'StateRte', 'MajorRd', 'Interstate', 'Road', 'Highway', 'USRte'}
basevals={'RailFeature', 'RoadSegment'}
mtypes={'Runway'}
basevals={'AirportRunway'}
#mtypes={'Rail'}
#basevals={'RailFeature'}

letters = string.ascii_uppercase
for mtype in mtypes:
    fnames = letters[:1]
    for fnum in range(len(fnames)):
        geomap   = {}
        records  = {}
        ngeos    = 0
        totalgeos = 0
        for state in states:
            if int(state) % len(fnames) != fnum:
                continue
            for baseval in basevals:
                try:
                    recname = "{0}/roads/{1}-{2}-{3}-{4}-data.p".format(basedir, baseval, mtype, state, prec)
                    recdata = pickle.load(open(recname, "rb"))
                except:
                    print("Could not open {0}".format(recname))
                    continue
                try:
                    geoname = "{0}/roads/{1}-{2}-{3}-{4}-geos.p".format(basedir, baseval, mtype, state, prec)
                    geodata = pickle.load(open(geoname, "rb"))
                except:
                    print("Could not open {0}".format(geoname))
                    continue
                ngeos = 0

                for geoid,geos in geodata.items():
                    ngeos += len(geos)
                    totalgeos += ngeos
                    for geo in geos:
                        if geomap.get(geo) is None:
                            geomap[geo] = Counter()
                        geomap[geo][mtype] += 1

                for geoid,rec in recdata.items():
                    records[geoid] = rec['Name']
                    break

        print("{0}\t{1}\t---> {2} <---".format(mtype,ngeos,len(geomap)))

        

        for geo in geomap.keys():
            geomap[geo] = dict(geomap[geo])
        
        if len(fnames) == 1:
            fname="geomap-{0}-{1}.p".format(prec, mtype)
        else:
            fname="geomap-{0}-{1}{2}.p".format(prec, mtype, fnames[fnum])            
        print("Writing {0}".format(fname))
        pickle.dump(geomap,  open(fname, "wb"))

        if len(fnames) == 1:
            fname="georec-{0}-{1}.p".format(prec, mtype)
        else:
            fname="georec-{0}-{1}{2}.p".format(prec, mtype, fnames[fnum])
        print("Writing {0}".format(fname))
        pickle.dump(records,  open(fname, "wb"))

Could not open /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-37-7-data.p
Could not open /Users/tgadf/Downloads/stateshapes/roads/AirportRunway-Runway-11-7-data.p
Runway	2467	---> 145627 <---
Writing geomap-7-Runway.p
Writing georec-7-Runway.p


# Plot Results

In [ ]:
from os.path import join
shapename = "/Users/tgadf/Downloads/stateshapes/TRAN_20_Kansas_GU_STATEORTERRITORY/Shape/Trans_RoadSegment"
#inputs = getShapeAndGeos(shapename, '1', debug=False)

In [ ]:
shapename = "/Users/tgadf/Downloads/stateshapes/TRAN_20_Kansas_GU_STATEORTERRITORY/Shape/Trans_RoadSegment"
shapedata = getRecordShape(shapename, '1', debug=True)

In [ ]:
sdata     = getShapeArray(shapedata, debug=True)
retval    = getGeos(shapedata, prec=8, linear=True, returnKeys=True, debug=True)
inputs    = {"Shape": sdata, "Geos": retval}

In [ ]:
import folium
center  = None
geoPnts = None

sdata   = inputs.get("Shape")
if sdata is None:
    print("Shape data is NONE!")
geos    = inputs.get("Geos")
if geos is None:
    print("Boundary geos data is NONE!")
intgeos = inputs.get("IntGeos")
if intgeos is None:
    print("Internal geos data is NONE!")

from geoClusteringPlots import drawClusters
dc = drawClusters()

if sdata is not None:
    print("Drawing shape with {0} points".format(sdata))
    from shapely.geometry.polygon import Polygon
    from shapely.geometry import Point
    try:
        poly      = Polygon(sdata)
        polyarea  = poly.area
    except:
        poly      = None
        
    if poly is not None:
        try:
            center    = poly.representative_point()
            center    = (center.x, center.y)
        except:
            center    = poly.centroid
            center    = (center.x, center.y)
    else:
        center = tuple(sdata[int(sdata.shape[0]/2),])

geoPnts=None
if geos is not None:
    geoPnts = []
    print("Drawing {0}".format(len(geos)))
    for geo in geos:
        geoPnts.append(dc.getBB(geo, istuple=True))

intGeoPnts=None
if intgeos is not None:
    intGeoPnts = []
    print("Drawing {0}".format(len(intgeos)))
    for geo in intgeos:
        intGeoPnts.append(dc.getBB(geo, istuple=True))

extraGeos=[]
extraGeoPnts=[]
for geo in extraGeos:
    extraGeoPnts.append(dc.getBB(geo, istuple=True))
        
if center is not None:
    m = folium.Map(location=center, zoom_start=11)

    if sdata is not None:
        folium.PolyLine(sdata, color='blue', weight=1, opacity=1).add_to(m)
    if geoPnts is not None:
        folium.PolyLine(geoPnts, color='red', weight=1, opacity=1).add_to(m)
    if intGeoPnts is not None:
        folium.PolyLine(intGeoPnts, color='purple', weight=1.25, opacity=1).add_to(m)
    if len(extraGeoPnts):
        folium.PolyLine(extraGeoPnts, color='green', weight=1.5, opacity=1).add_to(m)

    m.save("test.html")
m

In [ ]:
x = asarray([ [0, 0], [1, 1], [2,2] ])

In [ ]:
tuple(x[int(x.shape[0]/2),])

In [ ]:
def getGeoRecData(basedir, vals, prec):
    geomap  = {}
    records = {}    
    totalgeos = 0
    for mtype in vals:
        try:
            recdata = pickle.load(open("{0}/{1}-{2}-data.p".format(basedir, mtype, prec), "rb"))
            geodata = pickle.load(open("{0}/{1}-{2}-geos.p".format(basedir, mtype, prec), "rb"))
        except:
            print("Could not open files with base {0}/{1}-{2}".format(basedir, mtype, prec))
            continue
        #print("--> {0} {1}".format(len(recdata), len(geodata)))
        ngeos = 0
        for geoid,dummy in geodata.items():
            for gdummy, geos in dummy.items():
                ngeos += len(geos)
                totalgeos += ngeos
                for geo in geos:
                    if geomap.get(geo) is None:
                        geomap[geo] = {}
                    geomap[geo][mtype] = geoid

        for geoid,rec in recdata.items():
            records[geoid] = mtype
            #records[geoid] = rec['Name']
            break

        print("{0}\t{1}\t---> {2} <---".format(mtype,ngeos,len(geomap)))
        
    print("Found {0} total geohashs".format(totalgeos))
    return geomap, records